# HONR 39900 Fall 2021: Foundations of Geospatial Analytics
## Week 4 Class Notebook
## Basic PostGIS

### Justin A. Gould (gould29@purdue.edu)

# Required Packages

In [13]:
import geopandas as gpd
from sqlalchemy import create_engine, text
from sqlalchemy_utils import create_database, database_exists, drop_database

# Load Data (from week 3's London Borough Data)

In [2]:
map_df = gpd.read_file("../Week 3/ESRI/London_Borough_Excluding_MHW.shp")
map_df = map_df.to_crs(epsg=4326)

In [3]:
map_df.head()

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((-0.33068 51.32901, -0.33059 51.32909..."
1,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((-0.06402 51.31864, -0.06408 51.31861..."
2,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((0.01213 51.29960, 0.01196 51.29980, ..."
3,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((-0.24456 51.48870, -0.24468 51.48868..."
4,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((-0.41183 51.53408, -0.41188 51.53412..."


# Establishing the Connection to PostreSQL

- You need to make sure Postgres is running:
  - Via GUI (pgAdmin)
  - Postgres.app (Mac) - check upper right-hand corner
  - Via CLI:
  ```
  psql -U USERNAME -d DEMO
  ```

- If you followed the default suggestions I provided in lecture, once Postgres is running on your machine, the following cell _should_ work once you **change the password.**

In [5]:
#Variables
db_type = "postgres"
username = "postgres"
password = ""
host = "localhost"
port = "5432"
db_name = "demo"

#Put it together
engine = create_engine(f"{db_type}://{username}:{password}@{host}:{port}/{db_name}")

# Write Map to PostgreSQL

In [6]:
#Write map_df to PostgreSQL
map_df.to_postgis(name="london", con=engine)

In [10]:
engine.table_names() #We see that "london" was added to our database

['spatial_ref_sys', 'london']

# Spatial Queries via Postgres and PostGIS

## Accessing our Data

Before we can do any cool queries and geospatial transformations, we first need to be able to access our data.

In [27]:
#SQL query
sql = "SELECT * FROM london"

#Specify name of column which stores our geometry! In table `london`, the geometry is stored in a col called `geometry`
geom_col = "geometry"

#Execute query to create GeoDataFrame
map_df_from_db = gpd.GeoDataFrame.from_postgis(sql=sql, con=engine, geom_col=geom_col)

In [26]:
map_df_from_db.head() #Yay!

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((-0.33068 51.32901, -0.33059 51.32909..."
1,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((-0.06402 51.31864, -0.06408 51.31861..."
2,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((0.01213 51.29960, 0.01196 51.29980, ..."
3,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((-0.24456 51.48870, -0.24468 51.48868..."
4,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((-0.41183 51.53408, -0.41188 51.53412..."


## Basic PostGIS Functionality

When we use PostGIS and calculate spatial data/results, we save them as new columns via SQL.

In [30]:
#SQL Query to calculate area of borough polyon
sql = """
SELECT
    *,
    ST_Area(geometry) AS poly_area
FROM
    london
"""
gpd.GeoDataFrame.from_postgis(sql=sql, con=engine, geom_col=geom_col).head()

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry,poly_area
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((-0.33068 51.32901, -0.33059 51.32909...",0.004813
1,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((-0.06402 51.31864, -0.06408 51.31861...",0.011164
2,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((0.01213 51.29960, 0.01196 51.29980, ...",0.019384
3,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((-0.24456 51.48870, -0.24468 51.48868...",0.007237
4,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((-0.41183 51.53408, -0.41188 51.53412...",0.007196


Uh oh! What do the numbers under `poly_area` mean?

**HINT: Think back to our original London borough issue! It involves EPSG codes...**

Let's convert to sqft for a sphere, instead of a spheroid:

In [35]:
#SQL Query to calculate area of borough polyon (sqft for sphere)
sql = """
SELECT
    *,
    ST_Area(geometry) AS poly_area_orig,
    (ST_Area(geometry, false)) / (0.3048 ^ 2) AS poly_area_sqft,
    ((ST_Area(geometry, false)) / (0.3048 ^ 2)) / (27878400) AS poly_area_sqmiles
FROM
    london
"""
gpd.GeoDataFrame.from_postgis(sql=sql, con=engine, geom_col=geom_col).head()

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry,poly_area_orig,poly_area_sqft,poly_area_sqmiles
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((-0.33068 51.32901, -0.33059 51.32909...",0.004813,3.997146e+08,14.337787
1,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((-0.06402 51.31864, -0.06408 51.31861...",0.011164,9.278314e+08,33.281372
2,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((0.01213 51.29960, 0.01196 51.29980, ...",0.019384,1.616198e+09,57.973113
3,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((-0.24456 51.48870, -0.24468 51.48868...",0.007237,5.998885e+08,21.518039
4,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((-0.41183 51.53408, -0.41188 51.53412...",0.007196,5.956614e+08,21.366414


Let's check our work...

Our `london` table has borugh area in hectares! We can quickly convert to square miles to see how accurate we are.

In [46]:
#SQL Query to calculate area of borough polyon FROM HECTARES
sql = """
SELECT
    *,
    ST_Area(geometry)                                        AS poly_area_orig,
    (ST_Area(geometry, false)) / (0.3048 ^ 2)                AS poly_area_sqft,
    ((ST_Area(geometry, false)) / (0.3048 ^ 2)) / (27878400) AS poly_area_sqmiles,
    "HECTARES" * 0.0038610                                   AS sqmi_area_hectares --Wrap in double quotes
FROM
    london
"""
gpd.GeoDataFrame.from_postgis(sql=sql, con=engine, geom_col=geom_col).head()

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry,poly_area_orig,poly_area_sqft,poly_area_sqmiles,sqmi_area_hectares
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((-0.33068 51.32901, -0.33059 51.32909...",0.004813,3.997146e+08,14.337787,14.386538
1,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((-0.06402 51.31864, -0.06408 51.31861...",0.011164,9.278314e+08,33.281372,33.395492
2,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((0.01213 51.29960, 0.01196 51.29980, ...",0.019384,1.616198e+09,57.973113,57.967073
3,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((-0.24456 51.48870, -0.24468 51.48868...",0.007237,5.998885e+08,21.518039,21.847627
4,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((-0.41183 51.53408, -0.41188 51.53412...",0.007196,5.956614e+08,21.366414,21.445647


We are very close - look at the difference column below:

In [47]:
#SQL Query to calculate area of borough polyon FROM HECTARES
sql = """
SELECT
    *,
    ((ST_Area(geometry, false)) / (0.3048 ^ 2)) / (27878400) AS poly_area_sqmiles,
    "HECTARES" * 0.0038610                                   AS sqmi_area_hectares, --Wrap in double quotes
    ((ST_Area(geometry, false)) / (0.3048 ^ 2)) / (27878400) - "HECTARES" * 0.0038610 AS difference
FROM
    london
"""
gpd.GeoDataFrame.from_postgis(sql=sql, con=engine, geom_col=geom_col).head()

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry,poly_area_sqmiles,sqmi_area_hectares,difference
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((-0.33068 51.32901, -0.33059 51.32909...",14.337787,14.386538,-0.048751
1,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((-0.06402 51.31864, -0.06408 51.31861...",33.281372,33.395492,-0.114119
2,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((0.01213 51.29960, 0.01196 51.29980, ...",57.973113,57.967073,0.006039
3,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((-0.24456 51.48870, -0.24468 51.48868...",21.518039,21.847627,-0.329588
4,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((-0.41183 51.53408, -0.41188 51.53412...",21.366414,21.445647,-0.079232
